In [97]:
import pandas as pd
import numpy as np
import nltk
dat = pd.read_csv('amazon_co-ecommerce_sample.csv')
dat.head(5)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [98]:
# Keep only attributes that are important for the analysis
toys = dat[["product_name", "price", "average_review_rating", "customers_who_bought_this_item_also_bought", "customer_reviews"]]
# Change the display size
pd.set_option('display.max_columns',20)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
toys = toys.dropna(subset = ['average_review_rating', "customers_who_bought_this_item_also_bought", "customer_reviews"])
toys.reset_index(drop=True, inplace=True)
toys = pd.DataFrame(data=toys)
#print(toys)

In [28]:
""" # Function not used, to split the customer reviews into list
def separator(text):
    split_str = text.split('|') 
    return split_str
"""

In [29]:
"""# Remove '\n'
toys['customer_reviews'] = toys['customer_reviews'].apply(lambda x: x.replace('\n', ''))
toys['customer_reviews'] = toys['customer_reviews'].apply(separator)
toys.head()
"""

,product_name,price,average_review_rating,customers_who_bought_this_item_also_bought,customer_reviews
0,Hornby 2014 Catalogue,£3.42,4.9 out of 5 stars,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,[Worth Buying For The Pictures Alone (As Ever)...
1,FunkyBuys® Large Christmas Holiday Express Fes...,£16.99,4.5 out of 5 stars,http://www.amazon.co.uk/Christmas-Holiday-Expr...,[Four Stars // 4.0 // 18 Dec. 2015 // By ...
2,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,£9.99,3.9 out of 5 stars,http://www.amazon.co.uk/Classic-Train-Lights-B...,[**Highly Recommended!** // 5.0 // 26 May 2015...
3,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,£32.19,4.7 out of 5 stars,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,[Birthday present // 5.0 // 14 April 2014 // B...
4,20pcs Model Garden Light Double Heads Lamppost...,£6.99,5.0 out of 5 stars,http://www.amazon.co.uk/Single-Head-Garden-Lig...,[Five Stars // 5.0 // 27 Dec. 2014 // By ...


In [100]:
# Split data into mulitple rows
review_data = pd.concat([pd.Series(row['product_name'], row['customer_reviews'].split('|'))              
                    for _, row in toys.iterrows()]).reset_index()

In [101]:
review_data.head(5)

,index,0
0,Worth Buying For The Pictures Alone (As Ever) ...,Hornby 2014 Catalogue
1,Amazing detail fabulous photography. // 5.0 /...,Hornby 2014 Catalogue
2,'Great Purchase' // 5.0 // 23 April 2014 // B...,Hornby 2014 Catalogue
3,Great Catalogue // 5.0 // 11 Jun. 2014 // By\...,Hornby 2014 Catalogue
4,I collect them all as the glossy pictures are...,Hornby 2014 Catalogue


In [102]:
# Rename the columns
review_data.rename(columns={'index': "review"}, inplace=True)
review_data = review_data.rename(columns={ review_data.columns[1]: "product" })
review_data.head(10)

,review,product
0,Worth Buying For The Pictures Alone (As Ever) ...,Hornby 2014 Catalogue
1,Amazing detail fabulous photography. // 5.0 /...,Hornby 2014 Catalogue
2,'Great Purchase' // 5.0 // 23 April 2014 // B...,Hornby 2014 Catalogue
3,Great Catalogue // 5.0 // 11 Jun. 2014 // By\...,Hornby 2014 Catalogue
4,I collect them all as the glossy pictures are...,Hornby 2014 Catalogue
5,Great catalogue // 5.0 // 20 Mar. 2015 // By\...,Hornby 2014 Catalogue
6,Useful // 5.0 // 7 Oct. 2014 // By\n \n ...,Hornby 2014 Catalogue
7,hornbys latest catalogue. // 5.0 // 1 Dec. 20...,Hornby 2014 Catalogue
8,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,FunkyBuys® Large Christmas Holiday Express Fes...
9,Five Stars // 5.0 // 14 Jan. 2016 // By\n ...,FunkyBuys® Large Christmas Holiday Express Fes...


In [103]:
# Split into mulitple columns
split_data = review_data["review"].str.split("//", n = 4, expand = True)
split_data = split_data.rename(columns={ split_data.columns[0]: "title" , split_data.columns[1]: "rating", split_data.columns[2]: "date", split_data.columns[3]: "author",
                                        split_data.columns[4]: "review"})
# Append product
split_data['product'] = review_data['product']

In [115]:
split_data.head(10)

,title,rating,date,author,review,product
0,Worth Buying For The Pictures Alone (As Ever),4.0,6 April 2014,By\n \n Copnovelist\n \n on 6 April 2014,Part of the magic for me growing up as a boy ...,Hornby 2014 Catalogue
1,Amazing detail fabulous photography.,5.0,11 April 2015,By\n \n richard\n \n on 11 April 2015,"Amazing detail, every credit to the photograp...",Hornby 2014 Catalogue
2,'Great Purchase',5.0,23 April 2014,By\n \n Pinkhandbag\n \n on 23 April 2...,This was purchased on behalf of my Dad. He is...,Hornby 2014 Catalogue
3,Great Catalogue,5.0,11 Jun. 2014,By\n \n Gary John Mapson\n \n on 11 Ju...,Everything I really needed to see what was on...,Hornby 2014 Catalogue
4,I collect them all as the glossy pictures are...,5.0,7 Dec. 2014,By\n \n David Baker\n \n on 7 Dec. 2014,I collect them all as the glossy pictures are...,Hornby 2014 Catalogue
5,Great catalogue,5.0,20 Mar. 2015,By\n \n John A. Day\n \n on 20 Mar. 2015,What a great book. Extremely useful insight ...,Hornby 2014 Catalogue
6,Useful,5.0,7 Oct. 2014,By\n \n T. Davies\n \n on 7 Oct. 2014,Useful info for someonelike me starting back ...,Hornby 2014 Catalogue
7,hornbys latest catalogue.,5.0,1 Dec. 2014,By\n \n John Butlin\n \n on 1 Dec. 2014,A well produced very good quality catalogue.S...,Hornby 2014 Catalogue
8,Four Stars,4.0,18 Dec. 2015,By\n \n kenneth bell\n \n on 18 Dec. 2...,Very happy with the communication with funkyb...,FunkyBuys® Large Christmas Holiday Express Fes...
9,Five Stars,5.0,14 Jan. 2016,By\n \n moosixty\n \n on 14 Jan. 2016,Great buy.,FunkyBuys® Large Christmas Holiday Express Fes...


In [114]:
export_csv = split_data.to_csv('preprocess_review.csv', sep=',')

In [132]:
print(toys["customers_who_bought_this_item_also_bought"][0])

http://www.amazon.co.uk/Hornby-R8150-Catalogue-2015/dp/B00S9SUUBE | http://www.amazon.co.uk/Hornby-Book-Model-Railways-Edition/dp/1844860957 | http://www.amazon.co.uk/Hornby-Book-Scenic-Railway-Modelling/dp/1844861120 | http://www.amazon.co.uk/Peco-60-Plans-Book/dp/B002QVL16I | http://www.amazon.co.uk/Hornby-Gloucester | http://www.amazon.co.uk/Airfix-5014429781902


In [133]:
# Split customer_who_bought_this_also_bought
purchase_data = pd.concat([pd.Series(row['product_name'], row["customers_who_bought_this_item_also_bought"].split('|'))              
                    for _, row in toys.iterrows()]).reset_index()

In [134]:
purchase_data = purchase_data.rename(columns={ purchase_data.columns[0]: "links" , purchase_data.columns[1]: "product"})
purchase_data.head(10)
print(purchase_data['links'])

0        http://www.amazon.co.uk/Hornby-R8150-Catalogue...
1         http://www.amazon.co.uk/Hornby-Book-Model-Rai...
2         http://www.amazon.co.uk/Hornby-Book-Scenic-Ra...
3         http://www.amazon.co.uk/Peco-60-Plans-Book/dp...
4               http://www.amazon.co.uk/Hornby-Gloucester 
5             http://www.amazon.co.uk/Airfix-5014429781902
6        http://www.amazon.co.uk/Christmas-Holiday-Expr...
7         http://www.amazon.co.uk/Goldlok-Holiday-Expre...
8         http://www.amazon.co.uk/FunkyBuys%C2%AE-Chris...
9         http://www.amazon.co.uk/Holiday-Express-Chris...
10        http://www.amazon.co.uk/Seasonal-Vision-Chris...
11        http://www.amazon.co.uk/Coca-Cola-Santa-Expre...
12       http://www.amazon.co.uk/Classic-Train-Lights-B...
13        http://www.amazon.co.uk/Train-With-Tracks-Bat...
14        http://www.amazon.co.uk/13-Piece-Train-Set-Id...
15        http://www.amazon.co.uk/Train-Flash-Electric-...
16        http://www.amazon.co.uk/Train-Ultimate-Sticke.

In [135]:
purchase_data['links'] = purchase_data['links'].str.split("/", expand = False)
purchase_data.head(10)

,links,product
0,"[http:, , www.amazon.co.uk, Hornby-R8150-Catal...",Hornby 2014 Catalogue
1,"[ http:, , www.amazon.co.uk, Hornby-Book-Model...",Hornby 2014 Catalogue
2,"[ http:, , www.amazon.co.uk, Hornby-Book-Sceni...",Hornby 2014 Catalogue
3,"[ http:, , www.amazon.co.uk, Peco-60-Plans-Boo...",Hornby 2014 Catalogue
4,"[ http:, , www.amazon.co.uk, Hornby-Gloucester ]",Hornby 2014 Catalogue
5,"[ http:, , www.amazon.co.uk, Airfix-5014429781...",Hornby 2014 Catalogue
6,"[http:, , www.amazon.co.uk, Christmas-Holiday-...",FunkyBuys® Large Christmas Holiday Express Fes...
7,"[ http:, , www.amazon.co.uk, Goldlok-Holiday-E...",FunkyBuys® Large Christmas Holiday Express Fes...
8,"[ http:, , www.amazon.co.uk, FunkyBuys%C2%AE-C...",FunkyBuys® Large Christmas Holiday Express Fes...
9,"[ http:, , www.amazon.co.uk, Holiday-Express-C...",FunkyBuys® Large Christmas Holiday Express Fes...


In [141]:
# Extract the product name from the link
for i, row in purchase_data.iterrows():
    val = row["links"][3]
    purchase_data.at[i,'linked_prod'] = val
print(purchase_data['linked_prod'])

0                              Hornby-R8150-Catalogue-2015
1                       Hornby-Book-Model-Railways-Edition
2                     Hornby-Book-Scenic-Railway-Modelling
3                                       Peco-60-Plans-Book
4                                       Hornby-Gloucester 
5                                     Airfix-5014429781902
6          Christmas-Holiday-Express-Festive-Train-Set-Toy
7            Goldlok-Holiday-Express-Operated-Multi-Colour
8        FunkyBuys%C2%AE-Christmas-SI-TY1017-Ornaments-...
9           Holiday-Express-Christmas-Ornament-Decoration 
10                    Seasonal-Vision-Christmas-Tree-Train
11                       Coca-Cola-Santa-Express-Train-Set
12                   Classic-Train-Lights-Battery-Operated
13                    Train-With-Tracks-Battery-Operated-x
14                                13-Piece-Train-Set-Ideal
15                       Train-Flash-Electric-Sound-Europe
16                    Train-Ultimate-Sticker-Book-Sticke

In [142]:
export_csv = purchase_data.to_csv('preprocess_linked_products.csv', sep=',')